In [1]:
import sys
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
sys.path.append(".")
# os.environ["MAX_JOBS"] = "100"

import torch
torch.set_printoptions(linewidth=200, threshold=100000)
torch.cuda.is_available()

True

In [2]:
from src.vlstm_fwbw_v0.interface import vlstm_fw_torch, vlstm_fwbw_cuda
from src.vlstm_fwbw_v0.interface import vlstm_fw_cuda
from vlstm_formulations.vlstm_parallel import vlstm_parallel_fw_torch
# from vlstm_formulations.vlstm_parallel_tiled import vlstm_parallel_tiled
from src.vlstm_fwbw_v0.torch_impl import vlstm_fw_tiled_torch

INCLUDE: ['/home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include', '/home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/torch/csrc/api/include', '/home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/TH', '/home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/THC', '/home/max/miniconda3/envs/xlstmpt220cu121/include']
/home/max/myrepos/vlstm_cuda/src
/home/max/cpplibs/libtorch/lib:/usr/local/cuda-12.3/lib64:
/home/max/miniconda3/envs/xlstmpt220cu121/lib


Using /home/max/.cache/torch_extensions/py311_cu121 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/max/.cache/torch_extensions/py311_cu121/vlstm_fwbw_v0/build.ninja...
Building extension module vlstm_fwbw_v0...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


[1/2] /home/max/miniconda3/envs/xlstmpt220cu121/bin/nvcc --generate-dependencies-with-compile --dependency-output kernel_fw.cuda.o.d -ccbin /home/max/miniconda3/envs/xlstmpt220cu121/bin/x86_64-conda-linux-gnu-cc -DTORCH_EXTENSION_NAME=vlstm_fwbw_v0 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include -isystem /home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/torch/csrc/api/include -isystem /home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/TH -isystem /home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/THC -isystem /home/max/miniconda3/envs/xlstmpt220cu121/include -isystem /home/max/miniconda3/envs/xlstmpt220cu121/include/python3.11 -D_GLIBCXX_USE_CXX11_ABI=0 -D__CUDA_NO_HALF_OPERATORS__ -D__CUDA_NO_HALF_CONVERS

Loading extension module vlstm_fwbw_v0...


## CUDA vLSTM forward

In [3]:
S = 64 #32 #16 #8 # seq len
B = 1 # batch size
NH = 1 # num heads
DH = 8 # dim per head
DTYPE = torch.float32
DEVICE = torch.device("cuda:0")

In [4]:
# create qkv, inputgates, forgetgates 
torch.manual_seed(0)
qs = torch.arange((B*NH*S*DH), device=DEVICE, dtype=DTYPE).reshape((B, NH, S, DH)) / 10.
ks = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE) / 100.
vs = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE) / 100.
# igs = (1. + torch.arange((B * NH * S), device=DEVICE, dtype=DTYPE)).reshape(B, NH, S, 1) / 10.
# igs = torch.zeros((B, NH, S, 1), device=DEVICE, dtype=DTYPE) #/ 10.
igs = torch.randn((B, NH, S, 1), device=DEVICE, dtype=DTYPE) #/ 10.
fgs = torch.ones((B, NH, S, 1), device=DEVICE, dtype=DTYPE)
# fgs = torch.randn((B, NH, S, 1), device=DEVICE, dtype=DTYPE)
qs, qs.shape, len(qs.view(-1))

(tensor([[[[ 0.0000,  0.1000,  0.2000,  0.3000,  0.4000,  0.5000,  0.6000,  0.7000],
           [ 0.8000,  0.9000,  1.0000,  1.1000,  1.2000,  1.3000,  1.4000,  1.5000],
           [ 1.6000,  1.7000,  1.8000,  1.9000,  2.0000,  2.1000,  2.2000,  2.3000],
           [ 2.4000,  2.5000,  2.6000,  2.7000,  2.8000,  2.9000,  3.0000,  3.1000],
           [ 3.2000,  3.3000,  3.4000,  3.5000,  3.6000,  3.7000,  3.8000,  3.9000],
           [ 4.0000,  4.1000,  4.2000,  4.3000,  4.4000,  4.5000,  4.6000,  4.7000],
           [ 4.8000,  4.9000,  5.0000,  5.1000,  5.2000,  5.3000,  5.4000,  5.5000],
           [ 5.6000,  5.7000,  5.8000,  5.9000,  6.0000,  6.1000,  6.2000,  6.3000],
           [ 6.4000,  6.5000,  6.6000,  6.7000,  6.8000,  6.9000,  7.0000,  7.1000],
           [ 7.2000,  7.3000,  7.4000,  7.5000,  7.6000,  7.7000,  7.8000,  7.9000],
           [ 8.0000,  8.1000,  8.2000,  8.3000,  8.4000,  8.5000,  8.6000,  8.7000],
           [ 8.8000,  8.9000,  9.0000,  9.1000,  9.2000,  9.3000,

In [5]:
hs_pt, n_pt, m_pt, _, matD_pt = vlstm_fw_torch(queries=qs, keys=ks, values=vs, igate_preact=igs, fgate_preact=fgs)

In [6]:
# result_gt, m, l, n = vlstm_fw_tiled_torch(queries=qs, keys=ks, values=vs, igate_preact=igs, fgate_preact=fgs, bq_tile_size=8, bkv_tile_size=8)
# result_gt, result_gt.shape, torch.exp(-m), l, n

In [7]:
torch.nn.functional.logsigmoid(fgs).squeeze()[17] #+ (10.706538)
(fgs).squeeze()[17] #+ (10.706538)

tensor(1., device='cuda:0')

In [8]:
# cuda kernel
hs_cuda, n_cuda, m_cuda, matD_cuda = vlstm_fw_cuda(mat_Q=qs, mat_K=ks, mat_V=vs, igate_preact=igs.squeeze(-1), fgate_preact=fgs.squeeze(-1))

before kernel dispatch - float32!
B: 1, NH: 1, S: 64, DH: 8
blocksxy: 1-2, threadsxy: 4-4, shared_mem in bytes: 5700
In FW-Kernel: gdim.x: 1, gdim.y: 2, gdim.z: 1, bdim.x: 4, bdim.y: 4
In FW-Kernel: QtileDim: 8, KVtileDim: 8, TblockDim:4
IDX: cTileBlockY=0, qTileIdx=0, fChunkIdx=0 (<1), blockIdx.y=0, flatThreadIdx=0, kvTileIdx=0, InitFTileColVal=0.000000, fTileColLast=0.000000
IDX: cTileBlockY=16, qTileIdx=1, fChunkIdx=0 (<3), blockIdx.y=0, flatThreadIdx=0, kvTileIdx=0, InitFTileColVal=1.000000, fTileColLast=1.000000
IDX: cTileBlockY=16, qTileIdx=1, fChunkIdx=1 (<3), blockIdx.y=0, flatThreadIdx=0, kvTileIdx=0, InitFTileColVal=7.000000, fTileColLast=7.000000
IDX: cTileBlockY=16, qTileIdx=1, fChunkIdx=2 (<3), blockIdx.y=0, flatThreadIdx=0, kvTileIdx=0, InitFTileColVal=15.000000, fTileColLast=15.000000
IDX: cTileBlockY=32, qTileIdx=2, fChunkIdx=0 (<5), blockIdx.y=0, flatThreadIdx=0, kvTileIdx=0, InitFTileColVal=0.588397, fTileColLast=0.588397
IDX: cTileBlockY=32, qTileIdx=2, fChunkIdx=1 (

In [9]:
matD_cuda[:, :, 0:64, :16]

tensor([[[[ 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 1.,  0., -inf, -inf, -inf, -inf, -inf, -inf,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 2.,  1.,  0., -inf, -inf, -inf, -inf, -inf,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 3.,  2.,  1.,  0., -inf, -inf, -inf, -inf,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 4.,  3.,  2.,  1.,  0., -inf, -inf, -inf,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 5.,  4.,  3.,  2.,  1.,  0., -inf, -inf,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 6.,  5.,  4.,  3.,  2.,  1.,  0., -inf,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 7.,  6.,  5.,  4.,  3.,  2.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 8.,  7.,  6.,  5.,  4.,  3.,  2.,  1.,  0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [ 9.,  8.,  7.,  6.,  5.,  4.,  3.,  2.,  1.,  0., -inf, -inf, -inf, -inf, -inf, -inf],
          [10.,  9.,  8.,  7.,  6.,  5.

In [10]:
matD_pt[:, :, 0:64, :16]

tensor([[[[ 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [ 1.,  0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [ 2.,  1.,  0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [ 3.,  2.,  1.,  0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [ 4.,  3.,  2.,  1.,  0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [ 5.,  4.,  3.,  2.,  1.,  0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [ 6.,  5.,  4.,  3.,  2.,  1.,  0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [ 7.,  6.,  5.,  4.,  3.,  2.,  1.,  0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [ 8.,  7.,  6.,  5.,  4.,  3.,  2.,  1.,  0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [ 9.,  8.,  7.,  6.,  5.,  4.,  3.,  2.,  1.,  0., -inf, -inf, 

In [11]:
(matD_cuda - matD_pt)[:, :, :32, :10]

tensor([[[[0., nan, nan, nan, nan, nan, nan, nan, inf, inf],
          [0., 0., nan, nan, nan, nan, nan, nan, inf, inf],
          [0., 0., 0., nan, nan, nan, nan, nan, inf, inf],
          [0., 0., 0., 0., nan, nan, nan, nan, inf, inf],
          [0., 0., 0., 0., 0., nan, nan, nan, inf, inf],
          [0., 0., 0., 0., 0., 0., nan, nan, inf, inf],
          [0., 0., 0., 0., 0., 0., 0., nan, inf, inf],
          [0., 0., 0., 0., 0., 0., 0., 0., inf, inf],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., nan],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 

In [12]:
hs_cuda - hs_pt

tensor([[[[-7.2760e-12, -7.2760e-12, -7.2760e-12, -7.2760e-12, -7.2760e-12, -7.2760e-12, -7.2760e-12, -7.2760e-12],
          [ 1.1642e-10,  1.1642e-10,  1.1642e-10,  1.1642e-10,  1.1642e-10,  1.1642e-10,  1.1642e-10,  1.1642e-10],
          [-4.6566e-10, -4.6566e-10, -4.6566e-10, -4.6566e-10, -4.6566e-10, -4.6566e-10, -4.6566e-10, -4.6566e-10],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [-9.3132e-10, -9.3132e-10, -9.3132e-10, -9.3132e-10, -9.3132e-10, -9.3132e-10, -9.3132e-10, -9.3132e-10],
          [ 1.8626e-09,  1.8626e-09,  1.8626e-09,  1.8626e-09,  1.8626e-09,  1.8626e-09,  1.8626e-09,  1.8626e-09],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [-9.3132e-10, -9.3132e-10, -9.3132e-10, -9.3132e-10, -9.3132e-10, -9.3132e-10, -9.3132e-10, -9.3132e-10],
          [-9.3132e-10, -9.3132e-10, -9.3132e-10, -9.3132e-10, -9.3132e-

In [13]:
n_cuda.unsqueeze(-1) - n_pt

tensor([[[[ 0.0000e+00],
          [-2.9802e-08],
          [ 0.0000e+00],
          [-1.4901e-08],
          [-1.4901e-08],
          [-1.4901e-08],
          [-1.4901e-08],
          [ 0.0000e+00],
          [-5.9605e-08],
          [-2.9802e-08],
          [-5.9605e-08],
          [ 2.9802e-08],
          [-8.9407e-08],
          [-2.9802e-08],
          [ 0.0000e+00],
          [-5.9605e-08],
          [ 0.0000e+00],
          [-5.9605e-08],
          [-1.1921e-07],
          [-5.9605e-08],
          [-5.9605e-08],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [-5.9605e-08],
          [ 1.1921e-07],
          [-1.1921e-07],
          [ 0.0000e+00],
          [-5.9605e-08],
          [ 0.0000e+00],
          [ 1.1921e-07],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [-1.1921e-07],
          [ 0.0000e+00],
          [-1.1921e-07],
          [ 1.1921e-07],
          [-3.5763e-07],
          [-1.1921e-07],
          [-2.3842e-07],
          [ 0.0000e+00],


In [14]:
m_cuda.unsqueeze(-1) - m_pt

tensor([[[[0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
        

In [15]:
matD_cuda.shape

torch.Size([1, 1, 64, 64])

In [16]:
matD_cuda[:, :, :10, :10]

tensor([[[[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, 0., 0.],
          [1., 0., -inf, -inf, -inf, -inf, -inf, -inf, 0., 0.],
          [2., 1., 0., -inf, -inf, -inf, -inf, -inf, 0., 0.],
          [3., 2., 1., 0., -inf, -inf, -inf, -inf, 0., 0.],
          [4., 3., 2., 1., 0., -inf, -inf, -inf, 0., 0.],
          [5., 4., 3., 2., 1., 0., -inf, -inf, 0., 0.],
          [6., 5., 4., 3., 2., 1., 0., -inf, 0., 0.],
          [7., 6., 5., 4., 3., 2., 1., 0., 0., 0.],
          [8., 7., 6., 5., 4., 3., 2., 1., 0., -inf],
          [9., 8., 7., 6., 5., 4., 3., 2., 1., 0.]]]], device='cuda:0')

In [17]:
matD_pt

tensor([[[[ 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
           -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [ 1.,  0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
           -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [ 2.,  1.,  0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -

In [18]:
(matD_cuda - matD_pt)[:, :, :32, :10]

tensor([[[[0., nan, nan, nan, nan, nan, nan, nan, inf, inf],
          [0., 0., nan, nan, nan, nan, nan, nan, inf, inf],
          [0., 0., 0., nan, nan, nan, nan, nan, inf, inf],
          [0., 0., 0., 0., nan, nan, nan, nan, inf, inf],
          [0., 0., 0., 0., 0., nan, nan, nan, inf, inf],
          [0., 0., 0., 0., 0., 0., nan, nan, inf, inf],
          [0., 0., 0., 0., 0., 0., 0., nan, inf, inf],
          [0., 0., 0., 0., 0., 0., 0., 0., inf, inf],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., nan],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 

In [19]:
matD_pt[:, :, :10, :10]

tensor([[[[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [1., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [2., 1., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [3., 2., 1., 0., -inf, -inf, -inf, -inf, -inf, -inf],
          [4., 3., 2., 1., 0., -inf, -inf, -inf, -inf, -inf],
          [5., 4., 3., 2., 1., 0., -inf, -inf, -inf, -inf],
          [6., 5., 4., 3., 2., 1., 0., -inf, -inf, -inf],
          [7., 6., 5., 4., 3., 2., 1., 0., -inf, -inf],
          [8., 7., 6., 5., 4., 3., 2., 1., 0., -inf],
          [9., 8., 7., 6., 5., 4., 3., 2., 1., 0.]]]], device='cuda:0')